Both wm1 and m7 has been filtered at high frequency for wind at 230+/- 30 aggregated at 5 mins

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
from wind_tools import *  
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns
import matplotlib
#matplotlib.use('Qt5Agg')

import matplotlib.pyplot as plt
sns.set(style='whitegrid')
import plotly.express as px

matplotlib.rcParams['figure.figsize'] = (14,7)

In [ ]:
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize

## Load data

In [ ]:
proc_dir = Path("2020_data/data_field_v2_from_20208010/processed/filtered_2min")

# default is 2 mins aggregated data
wm1m_path = proc_dir / "eddypro_WM1_filtered_manual_full_output_2020-08-19T181014_exp.csv"
wm1ep_path = proc_dir / "eddypro_WM1_2min_filtered_EP_full_output_2020-08-19T125647_exp.csv"
m7_path = proc_dir / "eddypro_TRS_M507_filtered_full_output_2020-08-19T125615_exp.csv"



cache_dir = Path("data_cache")


wind_cols = ['u_unrot','v_unrot', 'w_unrot', 'wind_speed', 'wind_dir']
wind_comp_rename = {'u_unrot': 'u', 'v_unrot': 'v', 'w_unrot': 'w'}

In [ ]:
wm1m, wm1ep, m7= map(
    lambda p: load_ep_cache(p, cache_dir= cache_dir)
#     .loc[start_date:end_date]
    .loc[:,wind_cols]
    .rename(columns=wind_comp_rename),
    [wm1m_path, wm1ep_path, m7_path])

In [ ]:
wm1m

,u,v,w,wind_speed,wind_dir
date_time,,,,,
2020-08-10 14:32:00,0.686092,-1.70125,-0.432510,1.88467,249.302
2020-08-10 14:34:00,NaN,NaN,NaN,NaN,NaN
2020-08-10 14:36:00,1.025040,-2.30067,-0.167489,2.52426,246.769
2020-08-10 14:38:00,1.211080,-2.86820,-0.141769,3.11663,247.689
2020-08-10 14:40:00,0.698708,-1.68467,-0.074726,1.82534,248.737
...,...,...,...,...,...
2020-08-19 09:22:00,NaN,NaN,NaN,NaN,NaN
2020-08-19 09:24:00,NaN,NaN,NaN,NaN,NaN
2020-08-19 09:26:00,NaN,NaN,NaN,NaN,NaN


In [ ]:
wm1ep

,u,v,w,wind_speed,wind_dir
date_time,,,,,
2020-08-10 14:32:00,0.684925,-1.703860,-0.432726,1.885280,249.3410
2020-08-10 14:34:00,NaN,NaN,NaN,NaN,NaN
2020-08-10 14:36:00,1.024900,-2.297340,-0.167919,2.521190,246.7090
2020-08-10 14:38:00,1.197190,-2.843590,-0.135800,3.088320,248.0580
2020-08-10 14:40:00,0.651014,-1.606590,-0.061393,1.734570,250.6250
...,...,...,...,...,...
2020-08-19 09:22:00,NaN,NaN,NaN,NaN,NaN
2020-08-19 09:24:00,NaN,NaN,NaN,NaN,NaN
2020-08-19 09:26:00,NaN,NaN,NaN,NaN,NaN


In [ ]:
m7

,u,v,w,wind_speed,wind_dir
date_time,,,,,
2020-08-10 14:32:00,0.651861,-1.25005,-0.203992,1.42460,238.297
2020-08-10 14:34:00,NaN,NaN,NaN,NaN,NaN
2020-08-10 14:36:00,0.837281,-1.83572,-0.101121,2.02018,243.175
2020-08-10 14:38:00,1.052470,-2.37335,-0.020525,2.59633,244.146
2020-08-10 14:40:00,0.621153,-1.25263,-0.014319,1.39826,240.287
...,...,...,...,...,...
2020-08-19 09:22:00,NaN,NaN,NaN,NaN,NaN
2020-08-19 09:24:00,NaN,NaN,NaN,NaN,NaN
2020-08-19 09:26:00,NaN,NaN,NaN,NaN,NaN


In [ ]:
((wm1m != wm1ep) & wm1ep.isna()).sum()

u             0
v             0
w             0
wind_speed    0
wind_dir      0
dtype: int64

In [ ]:
(wm1m.isna() & wm1ep.isna()).sum()

u             3266
v             3266
w             3266
wind_speed    3266
wind_dir      3266
dtype: int64

In [ ]:
wm1ep.loc["2020-08-14 19:14:00"]

u            NaN
v            NaN
w            NaN
wind_speed   NaN
wind_dir     NaN
Name: 2020-08-14 19:14:00, dtype: float64

In [ ]:
wm1m.loc["2020-08-10 17:58:00"]

u            NaN
v            NaN
w            NaN
wind_speed   NaN
wind_dir     NaN
Name: 2020-08-10 17:58:00, dtype: float64

In [ ]:
m7.loc["2020-08-14 19:14:00"]

u               0.357802
v              -0.833121
w               0.082136
wind_speed      0.910417
wind_dir      246.444000
Name: 2020-08-14 19:14:00, dtype: float64

In [ ]:
for i in wm1m.index:
    if wm1ep.loc[i].isna().all() and not wm1m.loc[i].isna().all():
        print("non va...", i)

non va... 2020-08-14 19:14:00


In [ ]:
for i in wm1m.index:
    if wm1ep.loc[i].isna().all() and not m7.loc[i].isna().all():
        print("non va...", i)

non va... 2020-08-14 19:14:00


In [ ]:
wm1ep.isna().sum()

u             3267
v             3267
w             3267
wind_speed    3267
wind_dir      3267
dtype: int64

In [ ]:
wm1m.iloc[2] == wm1ep.iloc[2]

u             False
v             False
w             False
wind_speed    False
wind_dir      False
Name: 2020-08-10 14:36:00, dtype: bool

In [ ]:
wm1ep.describe()

,u,v,w,wind_speed,wind_dir
count,3063.000000,3063.000000,3063.000000,3063.000000,3063.000000
mean,0.191770,-0.882494,-0.049893,1.341664,169.570333
std,0.651685,1.143929,0.116347,0.874331,105.583920
min,-2.513480,-3.680150,-0.539933,0.031825,1.276100
25%,-0.248486,-1.938065,-0.126526,0.568986,41.936050
50%,0.280927,-0.894974,-0.034141,1.189470,247.497000
75%,0.723366,0.211128,0.028758,2.081775,251.521500
max,1.503660,1.851990,0.412975,3.975490,354.006000


In [ ]:
(wm1m.isna() == wm1ep.isna()).sum()

u             5218
v             5218
w             5218
wind_speed    5218
wind_dir      5218
dtype: int64

In [ ]:
wm1m.describe()

,u,v,w,wind_speed,wind_dir
count,1953.000000,1953.000000,1953.000000,1953.000000,1953.000000
mean,0.594865,-1.495669,-0.096977,1.724686,233.116495
std,0.383102,0.974331,0.108909,0.849498,52.000949
min,-0.516284,-3.680150,-0.547324,0.051851,13.929700
25%,0.333673,-2.199230,-0.167555,1.051170,246.957000
50%,0.646920,-1.701250,-0.089926,1.824660,249.680000
75%,0.864150,-0.942876,-0.018427,2.362180,252.034000
max,1.722730,1.877010,0.406538,3.975490,298.731000


In [ ]:
np.allclose(wm1m, wm1ep, 0.1)

False

In [ ]:
wm1.index

DatetimeIndex(['2020-07-24 13:05:00', '2020-07-24 13:10:00',
               '2020-07-24 13:15:00', '2020-07-24 13:20:00',
               '2020-07-24 13:25:00', '2020-07-24 13:30:00',
               '2020-07-24 13:35:00', '2020-07-24 13:40:00',
               '2020-07-24 13:45:00', '2020-07-24 13:50:00',
               ...
               '2020-08-07 08:15:00', '2020-08-07 08:20:00',
               '2020-08-07 08:25:00', '2020-08-07 08:30:00',
               '2020-08-07 08:35:00', '2020-08-07 08:40:00',
               '2020-08-07 08:45:00', '2020-08-07 08:50:00',
               '2020-08-07 08:55:00', '2020-08-07 09:00:00'],
              dtype='datetime64[ns]', name='date_time', length=3984, freq=None)

In [ ]:
m7.index

DatetimeIndex(['2020-07-24 13:05:00', '2020-07-24 13:10:00',
               '2020-07-24 13:15:00', '2020-07-24 13:20:00',
               '2020-07-24 13:25:00', '2020-07-24 13:30:00',
               '2020-07-24 13:35:00', '2020-07-24 13:40:00',
               '2020-07-24 13:45:00', '2020-07-24 13:50:00',
               ...
               '2020-08-07 08:15:00', '2020-08-07 08:20:00',
               '2020-08-07 08:25:00', '2020-08-07 08:30:00',
               '2020-08-07 08:35:00', '2020-08-07 08:40:00',
               '2020-08-07 08:45:00', '2020-08-07 08:50:00',
               '2020-08-07 08:55:00', '2020-08-07 09:00:00'],
              dtype='datetime64[ns]', name='date_time', length=3984, freq=None)

In [ ]:
m7.index

DatetimeIndex(['2020-07-24 13:05:00', '2020-07-24 13:10:00',
               '2020-07-24 13:15:00', '2020-07-24 13:20:00',
               '2020-07-24 13:25:00', '2020-07-24 13:30:00',
               '2020-07-24 13:35:00', '2020-07-24 13:40:00',
               '2020-07-24 13:45:00', '2020-07-24 13:50:00',
               ...
               '2020-08-06 23:15:00', '2020-08-06 23:20:00',
               '2020-08-06 23:25:00', '2020-08-06 23:30:00',
               '2020-08-06 23:35:00', '2020-08-06 23:40:00',
               '2020-08-06 23:45:00', '2020-08-06 23:50:00',
               '2020-08-06 23:55:00', '2020-08-07 00:00:00'],
              dtype='datetime64[ns]', name='date_time', length=3876, freq=None)

In [ ]:
plot_components([wm1, m7], ['wind_speed'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:title={'center':'wind_speed'}, xlabel='date_time'>],
      dtype=object)

In [ ]:
wm1r = rotate_wind_ang(wm1, 310)
wm1r = add_hor_wind_speed(wm1r)

# See amout of missing data

In [ ]:
preproc_dir = Path("2020_data/data_field_v2_from_20208010/preprocessed")

In [ ]:
empty = []
for f in (preproc_dir).glob("*_WM_174605_com1_filtered.csv"):
    df = pd.read_csv(f)
    empty.append([df.u.value_counts()[-9999], len(df)])

In [ ]:
nulls = pd.DataFrame(empty, columns=['n_null', 'total'])

In [ ]:
nulls = nulls.assign(ratio=nulls.n_null/nulls.total)

In [ ]:
nulls

,n_null,total,ratio
0,13200,18009,0.732967
1,14825,18012,0.823062
2,11562,18004,0.642191
3,15070,18013,0.836618
4,6369,18006,0.353715
...,...,...,...
409,17056,18012,0.946924
410,5569,18008,0.309251
411,13979,18011,0.776137
412,6879,18007,0.382018


In [ ]:
nulls.ratio.describe()

count    414.000000
mean       0.581790
std        0.219820
min        0.180834
25%        0.387343
50%        0.549352
75%        0.796047
max        0.961741
Name: ratio, dtype: float64

In [ ]:
ax=get_ax()
nulls.ratio.plot.hist(ax=ax, bins=20, density=False)
ax.set_xlabel("Ratio of discarded data/total data calculated for each half an hour")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Ratio of discarded data/total data calculated for each half an hour')

In [ ]:
nulls['cum_ratio'] = nulls['ratio'].cumsum()

In [ ]:
ax=get_ax()
nulls.ratio.hist(cumulative=True, ax=ax, bins=20, density=True)
ax.set_xlabel("Ratio of discarded data/total data calculated for each half an hour")
ax.set_ylabel("cumulative frequency")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'cumulative frequency')

In [ ]:
the_wm1 = pd.concat([pd.read_csv(f) for f in preproc_dir.glob("*_WM_174605_com1.csv")])

In [ ]:
the_wm1

,u,v,w,t
0,-0.08,-0.22,0.45,25.74
1,-0.05,-0.27,0.43,25.72
2,-0.22,-0.29,0.42,25.69
3,-0.20,-0.37,0.29,25.71
4,-0.06,-0.38,0.28,25.74
...,...,...,...,...
18007,-0.60,0.17,0.17,23.10
18008,-0.59,0.18,0.16,23.04
18009,-0.59,0.15,0.15,22.96
18010,-0.57,0.16,0.14,22.89


In [ ]:
the_wm1.memory_usage()

Index    59638808
u        59638808
v        59638808
w        59638808
t        59638808
dtype: int64

In [ ]:
the_wm1 = add_wind_dir(the_wm1)

In [ ]:
import matplotlib.patches as patches

In [ ]:
ax = get_ax()
g = sns.distplot(the_wm1.wind_dir, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:

ax.add_patch(
    patches.Rectangle((0, 220), 
                      60, 2, edgecolor='r'))


In [ ]:
pd.read_csv('2020_data/data_field_v2_from_20208010/processed/filtered_2min/../../preprocessed/20200813-0100_WM_174605_com1_filtered.csv').u.value_counts().index

Float64Index([-9999.0,     0.0,    0.01,    0.02,   -0.03,   -0.01,   -0.02,
                 0.03,   -0.04,    0.04,   -0.06,   -0.05,    0.06,    0.05,
                -0.09,   -0.07,   -0.08,    0.08,   -0.11,    0.07,    -0.1,
                -0.12,     0.1,    0.09,   -0.13,    0.11,    0.12,   -0.16,
                -0.15,    0.13,   -0.14,   -0.17,    0.14,   -0.19,   -0.18,
                 -0.2,   -0.21,   -0.22,    0.15,    0.17,    0.16,   -0.23,
                 0.18,   -0.24,    0.25,    0.28,    0.26,     0.2,    0.19,
                -0.25,    0.24,    0.23,    0.22,   -0.38,   -0.27,    -0.3,
                 0.21,    0.29,   -0.28,    0.31,   -0.26,   -0.29,    0.32,
                 0.27,   -0.31,    0.33,    0.34,   -0.36,    -0.4,   -0.35,
                  0.4,   -0.37,    0.37,   -0.39,    0.35,   -0.34,   -0.32,
                  0.3,   -0.42,   -0.45,   -0.43,    0.36,    0.38,   -0.33],
             dtype='float64')

# Load High freq data

In [ ]:
proc_dir = Path("2020_data/data_field_v2_from_20208010/preprocessed")
#manually filtered using custom code
wm1m_path = proc_dir / "20200810-1500_WM_174605_com1_filtered_250_30.csv"
# filtered using EddyPro
wm1ep_path = proc_dir / "../processed/filtered_2min/eddypro_raw_datasets/level_1" / "20200810-1504_raw_dataset_2020-08-13T090718_adv.txt"
wm1raw_path = proc_dir / "20200810-1500_WM_174605_com1.csv"

In [ ]:
proc_dir = Path("2020_data/data_field_v2_from_20208010/preprocessed")
#manually filtered using custom code
wm1m_path = proc_dir / "20200810-1430_WM_174605_com1_filtered_250_30.csv"
# filtered using EddyPro
wm1ep_path = proc_dir / "../processed/filtered_2min/eddypro_raw_datasets/level_1" / "20200810-1432_raw_dataset_2020-08-13T090718_adv.txt"
wm1raw_path = proc_dir / "20200810-1430_WM_174605_com1.csv"

In [ ]:
wm1m= pd.read_csv(wm1m_path)

# wm1m = wm1m[1199:2400-1].reset_index(drop=True)
wm1m = wm1m.drop(columns=['w', 't'])

In [ ]:
wm1ep = pd.read_csv(wm1ep_path, skiprows=9, delim_whitespace=True)

wm1ep = wm1ep.drop(columns=['w', 'ts'])

In [ ]:
wm1raw = pd.read_csv(wm1raw_path)
wm1raw = wm1raw.drop(columns=['w', 't'])
# wm1raw = wm1raw[1199:2400-1].reset_index(drop=True)

In [ ]:
# wm1m, wm1raw, wm1ep = map(lambda x: x.replace(-9999, np.nan), [wm1m, wm1raw, wm1ep])

## first exploration

In [ ]:
wm1raw

,u,v
0,1.09,-3.03
1,1.12,-3.46
2,1.71,-3.08
3,1.88,-3.00
4,2.05,-2.94
...,...,...
18002,0.13,-1.52
18003,0.21,-1.25
18004,-0.69,-1.34
18005,-0.79,-1.20


In [ ]:
wm1ep

,u,v
0,-9999.00,-9999.00
1,1.71,-3.08
2,1.88,-3.00
3,2.05,-2.94
4,2.00,-2.74
...,...,...
1194,-9999.00,-9999.00
1195,-9999.00,-9999.00
1196,-9999.00,-9999.00
1197,-9999.00,-9999.00


In [ ]:
wm1m

,u,v
0,-9999.0,-9999.0
1,-9999.0,-9999.0
2,-9999.0,-9999.0
3,-9999.0,-9999.0
4,-9999.0,-9999.0
...,...,...
18002,-9999.0,-9999.0
18003,-9999.0,-9999.0
18004,-9999.0,-9999.0
18005,-9999.0,-9999.0


In [ ]:
wm1m

,u,v
0,-9999.0,-9999.0
1,-9999.0,-9999.0
2,-9999.0,-9999.0
3,-9999.0,-9999.0
4,-9999.0,-9999.0
...,...,...
18002,-9999.0,-9999.0
18003,-9999.0,-9999.0
18004,-9999.0,-9999.0
18005,-9999.0,-9999.0


In [ ]:
get_wind_dir(wm1raw.u, wm1raw.v)

0        308.779050
1        304.721280
2        305.385431
3        305.487462
4        301.581292
            ...    
18002    310.689374
18003    310.044551
18004    328.535856
18005    330.875947
18006    319.763642
Length: 18007, dtype: float64

In [ ]:
wm1m[1199:2400-1]

,u,v
1199,-0.08,-1.81
1200,-0.04,-1.27
1201,0.29,-1.33
1202,0.95,-1.53
1203,0.61,-1.76
...,...,...
2394,1.56,-1.62
2395,1.36,-1.73
2396,1.50,-2.01
2397,1.16,-2.40


In [ ]:
(wm1m!=-9999).sum()

u    13
v    13
dtype: int64

In [ ]:
wm1m[wm1m!=-9999]

,u,v
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
1195,NaN,NaN
1196,NaN,NaN
1197,NaN,NaN
1198,NaN,NaN


In [ ]:
(wm1m==wm1raw).u.argmax()

280

In [ ]:
wm1m.loc[280:320]

,u,v
280,-0.16,-0.92
281,-0.16,-0.94
282,0.05,-1.05
283,-9999.00,-9999.00
284,-9999.00,-9999.00
285,-9999.00,-9999.00
286,-9999.00,-9999.00
287,-9999.00,-9999.00
288,-9999.00,-9999.00
289,-9999.00,-9999.00


In [ ]:
(wm1ep==-9999).sum()

u    312
v    312
dtype: int64

In [ ]:
np.allclose(wm1m, wm1ep)

True

In [ ]:
add_wind_dir(wm1m)

,u,v,wind_dir
0,-9999.0,-9999.0,315.0
1,-9999.0,-9999.0,315.0
2,-9999.0,-9999.0,315.0
3,-9999.0,-9999.0,315.0
4,-9999.0,-9999.0,315.0
...,...,...,...
1195,-9999.0,-9999.0,315.0
1196,-9999.0,-9999.0,315.0
1197,-9999.0,-9999.0,315.0
1198,-9999.0,-9999.0,315.0


In [ ]:
add_wind_dir(wm1ep)

,u,v,wind_dir
0,-0.08,-1.81,272.530763
1,-0.04,-1.27,271.803995
2,0.29,-1.33,257.699462
3,0.95,-1.53,238.163232
4,0.61,-1.76,250.884148
...,...,...,...
1195,1.56,-1.62,226.080925
1196,1.36,-1.73,231.828150
1197,1.50,-2.01,233.267173
1198,1.16,-2.40,244.203975


In [ ]:
add_wind_dir(wm1raw)

,u,v,wind_dir
0,-0.08,-1.81,272.530763
1,-0.04,-1.27,271.803995
2,0.29,-1.33,257.699461
3,0.95,-1.53,238.163232
4,0.61,-1.76,250.884149
...,...,...,...
1195,1.56,-1.62,226.080924
1196,1.36,-1.73,231.828150
1197,1.50,-2.01,233.267173
1198,1.16,-2.40,244.203974


In [ ]:
filt = filter_by_wind_dir_single(add_wind_dir(wm1raw).wind_dir, 250, 30) 

In [ ]:
wm1raw[~filt] = -9999

In [ ]:
wm1raw

,u,v
0,-0.08,-1.81
1,-0.04,-1.27
2,0.29,-1.33
3,0.95,-1.53
4,0.61,-1.76
...,...,...
1195,1.56,-1.62
1196,1.36,-1.73
1197,1.50,-2.01
1198,1.16,-2.40


In [ ]:
wm1ep

,u,v
0,-0.08,-1.81
1,-0.04,-1.27
2,0.29,-1.33
3,0.95,-1.53
4,0.61,-1.76
...,...,...
1195,1.56,-1.62
1196,1.36,-1.73
1197,1.50,-2.01
1198,1.16,-2.40


In [ ]:
np.allclose(wm1raw, wm1ep)

True

In [ ]:
mod(get_wind_dir(wm1ep.u, wm1ep.v) + 310)

0       265.000000
1       265.000000
2       253.151733
3       251.204748
4       249.872204
           ...    
1195    265.000000
1196    265.000000
1197    265.000000
1198    265.000000
1199    265.000000
Length: 1200, dtype: float64

In [ ]:
(wm1m != wm1raw).any()

u    False
v    False
dtype: bool

In [ ]:
wm1m.u.to_numpy()

array([-2.92, -2.78, -2.26, ..., -0.94, -0.97, -0.92])

In [ ]:
len(wm1raw)

1200

In [ ]:
mod( get_wind_dir(wm1m.u, wm1m.v) + 310)

0       265.000000
1       267.695551
2       253.151734
3       251.204747
4       249.872205
           ...    
1195    277.578935
1196    283.434949
1197    296.827447
1198    290.683647
1199    272.341226
Length: 1200, dtype: float64

In [ ]:
wm1m.isnull().any()

u    False
v    False
dtype: bool

In [ ]:
mod( get_wind_dir(wm1raw.u, wm1raw.v) + 310)

0       265.000000
1       267.695551
2       253.151734
3       251.204747
4       249.872205
           ...    
1195    277.578935
1196    283.434949
1197    296.827447
1198    290.683647
1199    272.341226
Length: 1200, dtype: float64

In [ ]:
filt = filter_by_wind_dir_single(mod( get_wind_dir(wm1m.u, wm1m.v) + 310), 230, 30) 

In [ ]:
wm1m[~filt] = -9999

In [ ]:
np.allclose(wm1ep, wm1m)

True

In [ ]:
wm1m

,u,v
0,NaN,NaN
1,NaN,NaN
2,-2.26,-3.46
3,-1.69,-2.79
4,-1.39,-2.42
...,...,...
1195,NaN,NaN
1196,NaN,NaN
1197,NaN,NaN
1198,NaN,NaN
